ANIMAL DETECTION MODEL

In [10]:
import pandas as pd
import os
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

Load animal information from CSV file

In [2]:
csv_path = r'C:\Users\SARATHLAL\Downloads\animal_det\animal_information.csv'
animal_info = pd.read_csv(csv_path)

Load images and labels

In [11]:
def load_images_and_labels(main_image_folder, animal_info):
    images = []
    labels = []
    for index, row in animal_info.iterrows():
        animal_name = row['Animal']
        diet = row['Diet']
        age_group = 'Adult' 
        animal_folder = os.path.join(main_image_folder, animal_name)
        if not os.path.exists(animal_folder):
            continue
        for filename in os.listdir(animal_folder):
            img_path = os.path.join(animal_folder, filename)
            image = load_img(img_path, target_size=(224, 224))
            image = img_to_array(image)
            images.append(image)
            labels.append({'name': animal_name, 'diet': diet, 'age_group': age_group})
    return np.array(images), labels

main_image_folder = r'C:\Users\SARATHLAL\Downloads\animal_det\animals'  # Update with the actual path
images, labels = load_images_and_labels(main_image_folder, animal_info)


Preprocessing labels for training

In [12]:
def preprocess_labels(labels):
    animal_names = list(animal_info['Animal'].unique())
    diets = ['Herbivore', 'Carnivore', 'Insectivore', 'Omnivore', 'Nectar, Insects']
    age_groups = ['Child', 'Adult']
    name_to_idx = {name: i for i, name in enumerate(animal_names)}
    diet_to_idx = {diet: i for i, diet in enumerate(diets)}
    age_to_idx = {age: i for i, age in enumerate(age_groups)}
    name_labels = [name_to_idx[label['name']] for label in labels]
    diet_labels = [diet_to_idx[label['diet']] for label in labels]
    age_labels = [age_to_idx[label['age_group']] for label in labels]
    return np.array(name_labels), np.array(diet_labels), np.array(age_labels)

name_labels, diet_labels, age_labels = preprocess_labels(labels)

Verify the label values are within the correct range

In [13]:
num_name_classes = len(animal_info['Animal'].unique())
num_diet_classes = len(['Herbivore', 'Carnivore', 'Insectivore', 'Omnivore', 'Nectar, Insects'])
num_age_classes = len(['Child', 'Adult'])

assert np.all(name_labels < num_name_classes), "Error: Some name labels are out of range"
assert np.all(diet_labels < num_diet_classes), "Error: Some diet labels are out of range"
assert np.all(age_labels < num_age_classes), "Error: Some age labels are out of range"

Split data

In [14]:
X_train, X_val, y_train_names, y_val_names = train_test_split(images, name_labels, test_size=0.2, random_state=42)
_, _, y_train_diets, y_val_diets = train_test_split(images, diet_labels, test_size=0.2, random_state=42)
_, _, y_train_ages, y_val_ages = train_test_split(images, age_labels, test_size=0.2, random_state=42)

Define the model

In [15]:
base_model = EfficientNetB0(include_top=False, input_tensor=Input(shape=(224, 224, 3)))
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu')(x)

name_output = Dense(num_name_classes, activation='softmax', name='name_output')(x)
diet_output = Dense(num_diet_classes, activation='softmax', name='diet_output')(x)
age_output = Dense(num_age_classes, activation='softmax', name='age_output')(x)

model = Model(inputs=base_model.input, outputs=[name_output, diet_output, age_output])

Compile the model

In [16]:
model.compile(optimizer=Adam(), 
              loss={'name_output': 'sparse_categorical_crossentropy', 
                    'diet_output': 'sparse_categorical_crossentropy', 
                    'age_output': 'sparse_categorical_crossentropy'},
              metrics={'name_output': 'accuracy', 
                       'diet_output': 'accuracy', 
                       'age_output': 'accuracy'})

Train the model

In [17]:
model.fit(X_train, {'name_output': y_train_names, 'diet_output': y_train_diets, 'age_output': y_train_ages},
          validation_data=(X_val, {'name_output': y_val_names, 'diet_output': y_val_diets, 'age_output': y_val_ages}),
          epochs=10, batch_size=32)

Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 96s 3s/step - age_output_accuracy: 0.8649 - diet_output_accuracy: 0.6889 - loss: 3.7261 - name_output_accuracy: 0.5801 - val_age_output_accuracy: 1.0000 - val_diet_output_accuracy: 0.9091 - val_loss: 0.5360 - val_name_output_accuracy: 0.9545
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 54s 3s/step - age_output_accuracy: 1.0000 - diet_output_accuracy: 0.9755 - loss: 0.2303 - name_output_accuracy: 0.9644 - val_age_output_accuracy: 1.0000 - val_diet_output_accuracy: 0.9091 - val_loss: 1.5486 - val_name_output_accuracy: 0.8333
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 56s 3s/step - age_output_accuracy: 1.0000 - diet_output_accuracy: 0.9581 - loss: 0.1579 - name_output_accuracy: 0.9866 - val_age_output_accuracy: 1.0000 - val_diet_output_accuracy: 0.9621 - val_loss: 0.4608 - val_name_output_accuracy: 0.9545
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 59s 3s/step - age_output_accuracy: 0.9981 - diet_output_accuracy: 0.9920 - loss: 0.0704 - name_output_accuracy: 0.9937 - v

Save the model

In [18]:
model.save('animal_classification_model.h5')

Feature extraction for clustering

In [19]:
def extract_features(image_folder):
    model = EfficientNetB0(include_top=False, pooling='avg', input_shape=(224, 224, 3))
    features = []
    image_paths = []
    
    for subdir, dirs, files in os.walk(image_folder):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                img_path = os.path.join(subdir, file)
                image = load_img(img_path, target_size=(224, 224))
                image = img_to_array(image)
                image = np.expand_dims(image, axis=0)
                image = model.predict(image)
                features.append(image.flatten())
                image_paths.append(img_path)
    
    return np.array(features), image_paths

features, image_paths = extract_features(main_image_folder)

# Clustering
kmeans = KMeans(n_clusters=2, random_state=42)
clusters = kmeans.fit_predict(features)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━

In [20]:
# Label clusters
cluster_labels = {0: 'Adult', 1: 'Child'}  # Adjust based on manual inspection
labeled_data = [(image_paths[i], cluster_labels[cluster]) for i, cluster in enumerate(clusters)]
labeled_df = pd.DataFrame(labeled_data, columns=['image_path', 'age_group'])

Fine-tune model on clustered age groups

In [21]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_dataframe(
    labeled_df,
    x_col='image_path',
    y_col='age_group',
    target_size=(224, 224),
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_dataframe(
    labeled_df,
    x_col='image_path',
    y_col='age_group',
    target_size=(224, 224),
    class_mode='binary',
    subset='validation'
)


Found 4320 validated image filenames belonging to 2 classes.
Found 1080 validated image filenames belonging to 2 classes.


In [22]:
base_model = EfficientNetB0(include_top=False, input_shape=(224, 224, 3))
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1, activation='sigmoid')(x)

age_model = Model(inputs=base_model.input, outputs=x)
age_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

age_model.fit(train_generator, validation_data=validation_generator, epochs=10)


Epoch 1/10


C:\Users\SARATHLAL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


135/135 ━━━━━━━━━━━━━━━━━━━━ 470s 3s/step - accuracy: 0.6976 - loss: 0.5917 - val_accuracy: 0.3528 - val_loss: 0.8114
Epoch 2/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 455s 3s/step - accuracy: 0.8581 - loss: 0.3393 - val_accuracy: 0.6491 - val_loss: 0.6670
Epoch 3/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 455s 3s/step - accuracy: 0.8879 - loss: 0.2714 - val_accuracy: 0.4213 - val_loss: 1.5495
Epoch 4/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 457s 3s/step - accuracy: 0.9339 - loss: 0.1925 - val_accuracy: 0.4685 - val_loss: 1.2657
Epoch 5/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 483s 4s/step - accuracy: 0.9296 - loss: 0.1753 - val_accuracy: 0.6009 - val_loss: 1.2407
Epoch 6/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 467s 3s/step - accuracy: 0.9552 - loss: 0.1333 - val_accuracy: 0.6889 - val_loss: 0.9496
Epoch 7/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 470s 3s/step - accuracy: 0.9468 - loss: 0.1510 - val_accuracy: 0.6722 - val_loss: 0.9018
Epoch 8/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 445s 3s/step - accuracy: 0.9726 - loss: 0.0892 - val_accuracy: 0.616

GUI for uploading and displaying predictions

In [23]:
def upload_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        img = Image.open(file_path)
        img_resized = img.resize((224, 224))
        img_array = img_to_array(img_resized)
        img_array = np.expand_dims(img_array, axis=0)
        
        # Predict
        preds = model.predict(img_array)
        name_pred = np.argmax(preds[0])
        diet_pred = np.argmax(preds[1])
        age_pred = np.argmax(preds[2])
        
        name = list(animal_info['Animal'].unique())[name_pred]
        diet = ['Herbivore', 'Carnivore', 'Insectivore', 'Omnivore', 'Nectar, Insects'][diet_pred]
        age = ['Child', 'Adult'][age_pred]
        
        result_text.set(f"Animal: {name}\nDiet: {diet}\nAge Group: {age}")
        
        img_tk = ImageTk.PhotoImage(img_resized)
        panel.config(image=img_tk)
        panel.image = img_tk


In [25]:
root = tk.Tk()
root.title("Animal Classifier")

result_text = tk.StringVar()
result_label = tk.Label(root, textvariable=result_text, font=("Helvetica", 16))
result_label.pack(pady=20)

upload_btn = tk.Button(root, text="Upload Image", command=upload_image, font=("Helvetica", 14))
upload_btn.pack(pady=10)

panel = tk.Label(root)
panel.pack(pady=10)

root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
